### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; updated 10.31.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [68]:
run ../common_functions.ipynb

In [69]:
run ../double_alpha_functions.ipynb

## Functions to simulate data

In [70]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [71]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [72]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn) # corresponds to recip_rates
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
#     probs = []
#     EVa = []
#     EVb = []
#     EVc = []
#     EVd = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        #print('trial type:', t)
        #print('begin EVs:', EVs[t])
        #print('begin prob:', Probs[t])
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
        #print('participant choice: ', choice)
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
        #print('partner response:', response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
#         probs.append(Probs[t])
        #print('updated prob: ', Probs[t])
        # then update value
        EVs[t] = update_value(Probs[t]) 
#         EVa.append(EVs[t][0])
#         EVb.append(EVs[t][1])
#         EVc.append(EVs[t][2])
#         EVd.append(EVs[t][3])
        #print('updated EVs:', EVs[t])

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}#, 
            #'Prob_Recip': probs, 'EVa': EVa, 'EVb': EVb, 'EVc': EVc, 'EVd': EVd}    
    return data

In [74]:
sim_data(15,[.2,.3,2])

trial type: 0
cumprob: [0.25 0.5  0.75 1.  ]
random_number 0.09099807185672071
participant choice:  1
trial type: 0
cumprob: [0.25 0.5  0.75 1.  ]
random_number 0.49319259408004634
participant choice:  2
trial type: 1
cumprob: [0.25 0.5  0.75 1.  ]
random_number 0.897581877015562
participant choice:  4
trial type: 1
cumprob: [0.01925242 0.0831727  0.2953955  1.        ]
random_number 0.9971737041916312
participant choice:  4
trial type: 2
cumprob: [0.25 0.5  0.75 1.  ]
random_number 0.8152493959943782
participant choice:  4
trial type: 0
cumprob: [0.01925242 0.0831727  0.2953955  1.        ]
random_number 0.2701788351749165
participant choice:  3
trial type: 2
cumprob: [0.83532475 0.97340301 0.99622719 1.        ]
random_number 0.136924459964458
participant choice:  1
trial type: 1
cumprob: [0.00135716 0.01312532 0.11516861 1.        ]
random_number 0.8098283292641612
participant choice:  4
trial type: 1
cumprob: [1.44970501e-04 2.85450000e-03 5.34961826e-02 1.00000000e+00]
random_numb

{'Trial': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45],
 'Stim_Sequence': array([0, 0, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 1, 1, 2,
        1, 1, 0, 1, 2, 2, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 1, 2, 0, 2,
        0]),
 'Choice': [1,
  2,
  4,
  4,
  4,
  3,
  1,
  4,
  4,
  4,
  1,
  4,
  4,
  1,
  3,
  4,
  1,
  4,
  4,
  4,
  4,
  1,
  4,
  3,
  4,
  1,
  1,
  1,
  2,
  4,
  4,
  1,
  4,
  3,
  1,
  4,
  1,
  1,
  4,
  3,
  2,
  1,
  4,
  1,
  4],
 'Trustee_Response': [1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1],
 'Prob_Recip': [0.5,
  0.6,
  0.6,
  0.6799999999999999,
  0.35,
  0.

### Pull in best-fit parameters from participants

In [9]:
dt = pd.read_csv('../../../output/two_alpha_model_params.csv')
print(dt)

          id  alpha_gain  alpha_loss       beta      X.LLH
0   sub-2013    0.003498    0.002143  20.000000  46.155479
1   sub-2007    0.289955    0.206801   0.303044  51.566509
2   sub-2006    0.000418    0.000000  19.110026  60.782458
3   sub-2012    0.018243    1.000000   0.416436  47.261224
4   sub-2004    0.070699    0.131740   2.112863  32.966208
..       ...         ...         ...        ...        ...
58  sub-2009    0.000000    1.000000   0.071504  61.424165
59  sub-2037    0.001407    0.005477  20.000000  48.727043
60  sub-2023    0.007018    0.016642  20.000000  31.610172
61  sub-2022    0.000000    0.332208   0.034781  60.909432
62  sub-2036    0.734792    0.000000   0.097911  61.106887

[63 rows x 5 columns]


In [10]:
a_gains = dt['alpha_gain']
a_losses = dt['alpha_loss']
betas = dt['beta']
parent_sub = dt['id']
print(a_gains)

0     0.003498
1     0.289955
2     0.000418
3     0.018243
4     0.070699
        ...   
58    0.000000
59    0.001407
60    0.007018
61    0.000000
62    0.734792
Name: alpha_gain, Length: 63, dtype: float64


### Simulate data

In [11]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta'])
for p in range(len(betas)): # loop for each participant
    for q in range(1,11): # loop for each simulated participant - change to increase the # of sims
        dt = sim_data(15, [a_gains[p], a_losses[p], betas[p]])
        dt['Subject'] = parent_sub[p] + '_' + str(q)
        dt['Alpha_gain'] = [a_gains[p]] * 45
        dt['Alpha_loss'] = [a_losses[p]] * 45
        dt['Beta'] = [betas[p]] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])

#data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_2alpha_model_data.csv', index = False)

In [12]:
print(data)

   Trial Stim_Sequence Choice Trustee_Response      Subject  Alpha_gain  \
0      1             1      2                0   sub-2013_1    0.003498   
1      2             2      1                0   sub-2013_1    0.003498   
2      3             1      2                1   sub-2013_1    0.003498   
3      4             0      3                1   sub-2013_1    0.003498   
4      5             1      4                1   sub-2013_1    0.003498   
..   ...           ...    ...              ...          ...         ...   
40    41             0      4                1  sub-2036_10    0.734792   
41    42             1      4                1  sub-2036_10    0.734792   
42    43             1      2                1  sub-2036_10    0.734792   
43    44             1      2                1  sub-2036_10    0.734792   
44    45             2      3                0  sub-2036_10    0.734792   

    Alpha_loss       Beta  Prob_Recip  EVa        EVb        EVc        EVd  
0     0.002143  20.00